In [ ]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv  # 🔄 추가된 부분

# 🔐 OpenAI API 키 가져오기
load_dotenv()  # .env 파일에서 환경변수 읽기

api_key = os.getenv("OPENAI_API_KEY")  # 변수명 수정
if api_key is None:
    raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")


# 전역 변수 설정
conversation_chain = None
chat_history = []

def process_pdf(pdf_file_path):
    """PDF 파일을 처리하여 벡터 데이터베이스를 생성합니다."""
    global conversation_chain
    
    try:
        if pdf_file_path is None:
            return "❌ PDF 파일을 업로드해주세요."
        
        # PDF 파일 로드 및 처리
        loader = PyPDFLoader(pdf_file_path)
        documents = loader.load()
        
        if not documents:
            return "❌ PDF에서 텍스트를 추출할 수 없습니다. 다른 PDF 파일을 시도해주세요."
        
        # 문서를 작은 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            length_function=len
        )
        chunks = text_splitter.split_documents(documents)
        
        if not chunks:
            return "❌ PDF 내용을 분할할 수 없습니다. 텍스트가 충분한지 확인해주세요."
        
        # 임베딩 생성 및 벡터 저장소 생성
        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(chunks, embeddings)
        
        # 대화 메모리 및 검색 체인 생성
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=ChatOpenAI(temperature=0.7, model_name="gpt-4"),
            retriever=vectorstore.as_retriever(),
            memory=memory
        )
        
        # 전역 대화 기록 초기화
        global chat_history
        chat_history = []
        
        return "✅ PDF가 성공적으로 처리되었습니다. 이제 질문을 입력해주세요!"
    
    except Exception as e:
        return f"❌ PDF 처리 중 오류가 발생했습니다: {str(e)}"

def answer_query(query, history):
    """사용자의 질문에 답변합니다."""
    global conversation_chain, chat_history
    
    if not query.strip():
        return history
    
    if conversation_chain is None:
        return history + [(query, "⚠️ PDF 파일을 먼저 업로드하고 처리해주세요.")]
    
    # 대화 기록에 사용자 질문 추가
    history.append((query, ""))
    
    try:
        # LangChain을 통해 답변 생성
        response = conversation_chain.invoke({"question": query})
        answer = response["answer"]
        
        # 대화 기록 업데이트
        chat_history.append((query, answer))
        history[-1] = (query, answer)
        
    except Exception as e:
        history[-1] = (query, f"❌ 오류가 발생했습니다: {str(e)}")
    
    return history

def reset_chat():
    """대화 기록을 초기화합니다."""
    global chat_history
    chat_history = []
    return []

# 텍스트 필드 초기화 함수
def clear_text():
    return ""

# Gradio 인터페이스 생성
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📄 PDF 기반 질의응답 시스템")
    gr.Markdown("PDF 파일을 업로드하고, 내용에 대해 질문하세요!")
    
    with gr.Row():
        with gr.Column(scale=1):
            # 파일 업로드 컴포넌트
            pdf_input = gr.File(
                label="PDF 파일 업로드", 
                file_types=[".pdf"],
                type="filepath"  # 파일 경로를 직접 사용
            )
            upload_button = gr.Button("PDF 처리하기", variant="primary")
            status_text = gr.Textbox(label="상태", interactive=False)
            clear_button = gr.Button("대화 초기화", variant="secondary")
        
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                [], 
                elem_id="chatbot", 
                height=500,
                avatar_images=(None, "https://api.dicebear.com/7.x/bottts/svg?seed=gpt")
            )
            
            with gr.Row():
                query_input = gr.Textbox(
                    placeholder="PDF 내용에 대해 질문하세요...",
                    label="질문",
                    lines=2,
                    interactive=True,
                    elem_id="query_input"
                )
                submit_btn = gr.Button("전송", variant="primary")
    
    # 이벤트 연결
    upload_button.click(
        fn=process_pdf,
        inputs=[pdf_input],
        outputs=[status_text]
    )
    
    # 질문 제출 방법 1: 텍스트박스 엔터키
    query_input.submit(
        fn=answer_query,
        inputs=[query_input, chatbot],
        outputs=[chatbot]
    ).then(
        fn=clear_text,  # 응답 후 입력 필드 지우기
        inputs=None,
        outputs=[query_input]
    )
    
    # 질문 제출 방법 2: 전송 버튼
    submit_btn.click(
        fn=answer_query,
        inputs=[query_input, chatbot],
        outputs=[chatbot]
    ).then(
        fn=clear_text,  # 응답 후 입력 필드 지우기
        inputs=None,
        outputs=[query_input]
    )
    
    # 대화 초기화 버튼
    clear_button.click(
        fn=reset_chat,
        outputs=[chatbot]
    )

# 자바스크립트로 Enter 키 이벤트 추가
demo.load(js="""
function setupEnterKey() {
    const textbox = document.getElementById('query_input').querySelector('textarea');
    textbox.addEventListener('keydown', function(event) {
        if (event.key === 'Enter' && !event.shiftKey) {
            event.preventDefault();
            const submitButton = document.querySelector('button.primary');
            submitButton.click();
        }
    });
}

// DOM이 완전히 로드된 후 실행
if (document.readyState === 'complete' || document.readyState === 'interactive') {
    setTimeout(setupEnterKey, 1000);
} else {
    document.addEventListener('DOMContentLoaded', function() {
        setTimeout(setupEnterKey, 1000);
    });
}
""")

# 앱 실행
if __name__ == "__main__":
    demo.launch()

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
/tmp/ipykernel_16218/2716360779.py:55: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
